In [ ]:
!pip install -U langchain-community
!pip install youtube-transcript-api
!pip install pytube
!pip install tiktoken
!pip install -qU langchain-cohere
!pip install chromadb
!pip install lark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 84.9 MB/s eta 0:00:00
   ━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
import getpass
import os
import sys

import numpy as np
from langchain.chains import RetrievalQA
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.llms import Cohere
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders.youtube import TranscriptFormat

In [ ]:
if not os.environ.get("COHERE_API_KEY"):
  os.environ["COHERE_API_KEY"] = getpass.getpass("Enter API Key:")

embeddings = CohereEmbeddings(model="embed-english-v3.0")

Enter API Key:··········


In [ ]:
YOUTUBE_VIDEO_LINK = "https://youtu.be/_-eHOSq3oqI"

In [ ]:
loader = YoutubeLoader.from_youtube_url(
    YOUTUBE_VIDEO_LINK,
    language=["es"],
    transcript_format=TranscriptFormat.CHUNKS,
    chunk_size_seconds=30
)

In [ ]:
content = loader.load()

In [ ]:
len(content)

17

In [ ]:
content

[Document(metadata={'source': 'https://www.youtube.com/watch?v=_-eHOSq3oqI&t=0s', 'start_seconds': 0, 'start_timestamp': '00:00:00'}, page_content="- [Speaker] Before we get into tariffs,- - The heart of the Donald\nTrump economic plan. - [Speaker] We should talk\nabout the chicken war. - The chicken war is\na cluckin' good story. - [Speaker] In post World\nWar II, West Germany, people started eating a lot of chicken, specifically American chicken. Midway through 1962, US\nfarmers were on track to sell more than $50\nmillion worth, half a billion in today's money. This made European farmers mad. So the organization that later\nbecame the European Union"),
 Document(metadata={'source': 'https://www.youtube.com/watch?v=_-eHOSq3oqI&t=30s', 'start_seconds': 30, 'start_timestamp': '00:00:30'}, page_content="put a tariff on chicken A five pound chicken that\nstarted as $1.60 became $2.25. Imports quickly dropped. US chicken farmers and\npoliticians were furious. - And we thought, Germany's\n

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 150
)

In [ ]:
splits = text_splitter.split_documents(content)
len(splits)

17

In [ ]:
splits

[Document(metadata={'source': 'https://www.youtube.com/watch?v=_-eHOSq3oqI&t=0s', 'start_seconds': 0, 'start_timestamp': '00:00:00'}, page_content="- [Speaker] Before we get into tariffs,- - The heart of the Donald\nTrump economic plan. - [Speaker] We should talk\nabout the chicken war. - The chicken war is\na cluckin' good story. - [Speaker] In post World\nWar II, West Germany, people started eating a lot of chicken, specifically American chicken. Midway through 1962, US\nfarmers were on track to sell more than $50\nmillion worth, half a billion in today's money. This made European farmers mad. So the organization that later\nbecame the European Union"),
 Document(metadata={'source': 'https://www.youtube.com/watch?v=_-eHOSq3oqI&t=30s', 'start_seconds': 30, 'start_timestamp': '00:00:30'}, page_content="put a tariff on chicken A five pound chicken that\nstarted as $1.60 became $2.25. Imports quickly dropped. US chicken farmers and\npoliticians were furious. - And we thought, Germany's\n

In [ ]:
sentence1 = "How expensive it is going to be having a car?"
sentence2 = "Crude Oil is expected to trade at 59.23 USD/BBL by the end of this quarter, according to Trading Economics global macro models and analysts expectations."
sentence3 = "The Federal Reserve System (often shortened to the Federal Reserve, or simply the Fed) is the central banking system of the United States. It was created on December 23, 1913, with the enactment of the Federal Reserve Act, after a series of financial panics"

In [ ]:
embedding1 = embeddings.embed_query(sentence1)
embedding2 = embeddings.embed_query(sentence2)
embedding3 = embeddings.embed_query(sentence3)

In [ ]:
print("Calculating similarity between...")
print(sentence1)
print(sentence2)
np.dot(embedding1, embedding2)

Calculating similarity between...
How expensive it is going to be having a car?
Crude Oil is expected to trade at 59.23 USD/BBL by the end of this quarter, according to Trading Economics global macro models and analysts expectations.


np.float64(0.26424480251305743)

In [ ]:
print("Calculating similarity between...")
print(sentence1)
print(sentence3)
np.dot(embedding1, embedding3)

Calculating similarity between...
How expensive it is going to be having a car?
The Federal Reserve System (often shortened to the Federal Reserve, or simply the Fed) is the central banking system of the United States. It was created on December 23, 1913, with the enactment of the Federal Reserve Act, after a series of financial panics


np.float64(0.12680274771276878)

In [ ]:
persist_directory = 'docs/chroma/'

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [ ]:
question = "How has the last administration responded to Trump’s tariffs?"
docs = vectordb.max_marginal_relevance_search(question,k=3)

In [ ]:
docs

[Document(metadata={'start_timestamp': '00:04:30', 'source': 'https://www.youtube.com/watch?v=_-eHOSq3oqI&t=270s', 'start_seconds': 270}, page_content="But it didn't change\nmuch of China's behavior and in many ways they\nbecame more aggressive. But here's the thing, even with\nall the economic downsides, a review by the Biden administration suggested Trump's tariffs should remain and the administration should add or increase them, which they did. - Once tariffs get in place,\nthere are two factors that sort of keep them in place for some time. - First of all, they're gonna\nbe domestic interest groups that have a stake in seating\nthose tariffs maintained 'cause they like the extra\nmarket share that they've gained"),
 Document(metadata={'start_timestamp': '00:01:30', 'start_seconds': 90, 'source': 'https://www.youtube.com/watch?v=_-eHOSq3oqI&t=90s'}, page_content="- President Trump has\nreally sort of brought a lot of people out to think that\ntariffs might be something that's useful

In [ ]:
metadata_field_info = [
    AttributeInfo(
        name="timestamp",
        description="The time in the video in format HH:MM:SS",
        type="string",
    )
]

In [ ]:
document_content_description = "Why Economists Hate Trump's Tariff Plan | WSJ Wall Street Journal"
llm = Cohere(model="command-r-plus")
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

<ipython-input-33-e318c8c32776>:2: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm = Cohere(model="command-r-plus")


In [ ]:
llm.model

'command-r-plus'

In [ ]:
print("question", question)
docs = retriever.get_relevant_documents(question)
docs

question How has the last administration responded to Trump’s tariffs?


<ipython-input-35-8c9d69fcf25a>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


[Document(metadata={'start_seconds': 390, 'start_timestamp': '00:06:30', 'source': 'https://www.youtube.com/watch?v=_-eHOSq3oqI&t=390s'}, page_content="- They're not gonna take\nthat this lying down. They're not going to change\ntheir behavior in some way to appease the administration\nthat imposes such tariffs. They're going to retaliate. So not only will US import shrink, but US exports will shrink as well. - [Speaker] Trump wants\nto use these tariffs as a revenue stream for\nthe US to pay for tax cuts. - Whereas tariffs on\nforeign countries go up, taxes on American workers\nand families come down. - [Speaker] Studies found his\nproposal would likely bring in"),
 Document(metadata={'source': 'https://www.youtube.com/watch?v=_-eHOSq3oqI&t=270s', 'start_timestamp': '00:04:30', 'start_seconds': 270}, page_content="But it didn't change\nmuch of China's behavior and in many ways they\nbecame more aggressive. But here's the thing, even with\nall the economic downsides, a review by the Bi

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
result = qa_chain({"query": question})

<ipython-input-37-ef85128c39b8>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [ ]:
result["result"]

"The Biden administration reviewed Trump's tariffs and decided to keep and increase them."

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
result = qa_chain({"query": question})

In [ ]:
result["result"]

"The Biden Administration suggested Trump's tariffs should remain in place."

In [ ]:
start_seconds = docs[0].metadata["start_seconds"]
start_seconds

390

In [ ]:
from IPython.display import Markdown

link = YOUTUBE_VIDEO_LINK+"?t="+str(start_seconds)
Markdown(f"[{link}]({link})")


[https://youtu.be/_-eHOSq3oqI?t=390](https://youtu.be/_-eHOSq3oqI?t=390)